In [1]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx

from ema_workbench import (
    Model,
    Policy,
    Scenario,
    ema_logging,
    SequentialEvaluator,
    MultiprocessingEvaluator,
    Samplers,
)
from dike_model_function import DikeNetwork  # @UnresolvedImport
from problem_formulation import get_model_for_problem_formulation, sum_over, sum_over_time
dike_model, planning_steps = get_model_for_problem_formulation(5)



In [2]:
n_scenarios = 4096
n_policies = 4096

def get_do_nothing_dict():
    return {l.name: 0 for l in dike_model.levers}

policies = [
    Policy(
        "policy 0",
        **dict(
            get_do_nothing_dict()
        )
    ),]

In [3]:
# load dataframe from results folder
df = pd.read_csv("results/scenario_space_100000_experiments_2025-06-11_21-41.csv")

In [4]:
pd.set_option('display.max_columns', None)
df.describe()

,A.0_ID flood wave shape,A.1_Bmax,A.1_Brate,A.1_pfail,A.2_Bmax,A.2_Brate,A.2_pfail,A.3_Bmax,A.3_Brate,A.3_pfail,A.4_Bmax,A.4_Brate,A.4_pfail,A.5_Bmax,A.5_Brate,A.5_pfail,discount rate 0,discount rate 1,discount rate 2,0_RfR 0,0_RfR 1,0_RfR 2,1_RfR 0,1_RfR 1,1_RfR 2,2_RfR 0,2_RfR 1,2_RfR 2,3_RfR 0,3_RfR 1,3_RfR 2,4_RfR 0,4_RfR 1,4_RfR 2,EWS_DaysToThreat,A.1_DikeIncrease 0,A.1_DikeIncrease 1,A.1_DikeIncrease 2,A.2_DikeIncrease 0,A.2_DikeIncrease 1,A.2_DikeIncrease 2,A.3_DikeIncrease 0,A.3_DikeIncrease 1,A.3_DikeIncrease 2,A.4_DikeIncrease 0,A.4_DikeIncrease 1,A.4_DikeIncrease 2,A.5_DikeIncrease 0,A.5_DikeIncrease 1,A.5_DikeIncrease 2,scenario
count,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,1.000000e+05,100000.00000,100000.00000,100000.00000,100000.0,100000.0,100000.0,100000.0,100000.0,100000.0,100000.0,100000.0,100000.0,100000.0,100000.0,100000.0,100000.0,100000.0,100000.0,100000.0,100000.0,100000.0,100000.0,100000.0,100000.0,100000.0,100000.0,100000.0,100000.0,100000.0,100000.0,100000.0,100000.0,100000.0,100000.0,100000.000000
mean,66.000030,190.000003,4.166640,0.500000,190.000001,4.166640,0.500000,190.000000,4.166720,0.500000,189.999996,4.166720,0.500000,189.999999,4.166640,5.000000e-01,3.00000,3.00000,3.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,49999.500000
std,38.392878,92.376504,4.129846,0.288677,92.376508,4.129846,0.288677,92.376506,4.129882,0.288677,92.376509,4.129882,0.288677,92.376507,4.129846,2.886766e-01,1.11804,1.11804,1.11804,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,28867.657797
min,0.000000,30.000631,1.000000,0.000001,30.000995,1.000000,0.000006,30.002090,1.000000,0.000002,30.001145,1.000000,0.000009,30.002744,1.000000,1.479923e-07,1.50000,1.50000,1.50000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
25%,33.000000,110.001034,1.000000,0.250002,110.001342,1.000000,0.250002,109.999956,1.000000,0.250006,110.001812,1.000000,0.250002,110.000467,1.000000,2.499999e-01,2.25000,2.25000,2.25000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,24999.750000
50%,66.000000,189.998981,1.500000,0.500000,190.000630,1.500000,0.499998,190.000919,1.500000,0.500004,189.999591,1.500000,0.500001,190.000754,1.500000,4.999984e-01,3.00000,3.00000,3.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,49999.500000
75%,99.000000,269.998640,10.000000,0.750000,269.998418,10.000000,0.749998,270.000085,10.000000,0.749998,269.999752,10.000000,0.749996,269.998503,10.000000,7.499969e-01,3.75000,3.75000,3.75000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,74999.250000
max,132.000000,349.999024,10.000000,0.999999,349.997008,10.000000,0.999991,349.999395,10.000000,0.999994,349.996943,10.000000,1.000000,349.998154,10.000000,9.999954e-01,4.50000,4.50000,4.50000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,99999.000000


In [5]:
scenario = None
if scenario is None:
        reference_values = {
            "Bmax": 190,
            "Brate": 4.167,
            "pfail": 0.5,
            "discount rate 0": 3,
            "discount rate 1": 3,
            "discount rate 2": 3,
            "ID flood wave shape": 4, # A bit arbitrary tbh
        }
        scenario_dict = {}
        for key in dike_model.uncertainties:
            name_split = key.name.split("_")
            if len(name_split) == 1:
                scenario_dict[key.name] = reference_values.get(key.name)
            else:
                scenario_dict[key.name] = reference_values.get(name_split[1])
        scenario = Scenario("default", **scenario_dict)
scenarios = [scenario,]


In [6]:
import os
from datetime import datetime
import pandas as pd

def save_experiments_and_outcomes(name, experiments, outcomes, folder_name="sobol_results"):
    """
    Saves the experiments and outcomes to CSV files in the specified folder,
    with filenames that include a custom name and timestamp.

    Parameters:
    - name (str): Custom label for the output files (e.g., 'policy6_seed20').
    - experiments (pd.DataFrame): The experiments DataFrame.
    - outcomes (dict): The outcomes dictionary from the EMA Workbench.
    - folder_name (str): The name of the folder where files will be saved. Default is "results".
    """
    # Get timestamp
    current_date = datetime.now().strftime('%Y-%m-%d')

    # Ensure folder exists
    os.makedirs(folder_name, exist_ok=True)

    # Define file paths
    exp_path = os.path.join(folder_name, f'{name}_SOBOL_uncertainty_{current_date}.csv')
    out_path = os.path.join(folder_name, f'{name}_SOBOL_policy_{current_date}.csv')

    # Save experiments
    experiments.to_csv(exp_path, index=False)

    # Flatten outcomes and save
    flattened_data = {key: value.flatten() for key, value in outcomes.items()}
    df_outcomes = pd.DataFrame(flattened_data)
    df_outcomes.to_csv(out_path, index=False)

    print(f"Saved experiments and outcomes as '{name}' in '{folder_name}' (timestamp: {current_date})")


Jesse

In [8]:
# running the model through EMA workbench
with MultiprocessingEvaluator(dike_model) as evaluator:
    results = evaluator.perform_experiments(scenarios=n_scenarios, policies=policies, uncertainty_sampling=Samplers.SOBOL)
experiments, outcomes = results
save_experiments_and_outcomes('experiments_outcomes', experiments, outcomes)
# 10 takes 1 minute --> 1000 --> 3 hours

100%|██████████████████████████████████████████| 40/40 [00:06<00:00,  6.53it/s]


Saved experiments and outcomes as 'experiments_outcomes' in 'sobol_results' (timestamp: 2025-06-17)


Twan

In [9]:
with MultiprocessingEvaluator(dike_model) as evaluator:
    results = evaluator.perform_experiments(scenarios=scenarios, policies=n_policies, lever_sampling=Samplers.SOBOL)   
experiments, outcomes = results
save_experiments_and_outcomes('experiments_outcomes', experiments, outcomes)

100%|██████████████████████████████████████████| 64/64 [00:08<00:00,  7.76it/s]


Saved experiments and outcomes as 'experiments_outcomes' in 'sobol_results' (timestamp: 2025-06-17)
